# Analyze multiple video and compare results
Nikola Bátová & Jesse ten Broeke

n.batova@student.rug.nl

j.ten.broeke1@student.rug.nl

In [44]:
from functions_for_DLC_analysis import get_bodyparts, get_center_coordinates, get_radius, create_rois, get_total_exploration_time, get_cm_to_pixel_ratio, get_filenames
# Importing the toolbox (takes several seconds)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Have this file in the same folder as the notebook
import time_in_each_roi
import Time_within_certain_dist

#import image_draw
import matplotlib
matplotlib.use('Agg')

from collections import namedtuple
from matplotlib.patches import Circle
import copy
import cv2 as cv
import os

In [45]:
# Variable for the Dataframe to locate .h5 data
DLCscorer = 'DLC_resnet50_BN research projectFeb3shuffle1_54500'

video_data = pd.read_csv(f'video_data.csv')

video_names = video_data['videoname']
cm_to_pixel_ratios = (float(i) for i in video_data['cm_to_pixel_ratio'])

# Loading DLC data to analyse
dfs = []
for video in video_names:
    df= pd.read_hdf(f'videos/{video}DLC_resnet50_BN research projectFeb3shuffle1_54500.h5')
    dfs.append(df)

In [46]:
# Bugfix to take care of numpy nep50 warning.
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [47]:
# Frames per seconds in the videos being scored (default 30)
fps=30

# ADD body parts whose time you want to track in your ROI
bpt_plus_names=['nose']

# ADD body parts whose time you want to substract from the total time
bpt_minus_names=['left-front-paw', 'right-front-paw']


In [48]:
# ADD a list of names of ROIs
rois_names = ['non-moved object', 'moved object']

# ADD how many cm from your object you want to track you body part, default 1
boundary_size_distance = 1

In [ ]:
results = []
for video, ratio in zip(video_names, cm_to_pixel_ratios):
    print(f'Video {video}:')

    df= pd.read_hdf(f'videos/{video}DLC_resnet50_BN research projectFeb3shuffle1_54500.h5')

    rois = create_rois(df=df, DLCscorer=DLCscorer, distance=boundary_size_distance, names=rois_names, cm_to_pixel_ratio=ratio)

    # Create rois for which time will be substracted if a certain body part is in one of these ROIs
    rois_substracting = create_rois(df=df, DLCscorer=DLCscorer, distance=0, names=rois_names, cm_to_pixel_ratio=ratio)

    # Ectract time, velocity & x and y coordinates for each body part
    bpt_plus = get_bodyparts(df=df, DLCscorer=DLCscorer, fps=fps, bpt_names=bpt_plus_names)

    # Ectract time, velocity & x and y coordinates for each body part
    bpt_minus = get_bodyparts(df=df, DLCscorer=DLCscorer, fps=fps, bpt_names=bpt_minus_names)

    exploration = get_total_exploration_time(rois, rois_substracting, bpt_plus, bpt_minus)
    #print(exploration)
    for exp in exploration:
        print(f"Total Exploration Time for {exp.get('name')}: {exp.get('exploration time')}s")
        results.append({
            "video_name": video,
            "roi_name": exp.get("name"),
            "exploration_time": exp.get("exploration time"),
            "cm_to_pixel_ratio": ratio
        })
    print("\n")
    

Video 233 (1):
Ration 20.8262154507246:
Total Exploration Time for non-moved object: 14.93333333333333s
Total Exploration Time for moved object: 8.6s


Video 234 (1):
Ration 19.45006426724601:
Total Exploration Time for non-moved object: 38.766666666666666s
Total Exploration Time for moved object: 1.5666666666666667s


Video 235 (1):
Ration 19.556392816672503:
Total Exploration Time for non-moved object: 14.433333333333335s
Total Exploration Time for moved object: 20.9s


Video 236 (1):
Ration 20.950238662125063:
Total Exploration Time for non-moved object: 1.2999999999999998s
Total Exploration Time for moved object: 9.699999999999998s


Video 237 (1):
Ration 19.87514150893019:
Total Exploration Time for non-moved object: 3.099999999999999s
Total Exploration Time for moved object: 22.666666666666668s


Video 238 (1):
Ration 19.23773960214661:
Total Exploration Time for non-moved object: 29.433333333333334s
Total Exploration Time for moved object: 5.766666666666668s


Video 239 (1):
Rat

In [50]:
# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save to CSV
df_results.to_csv("exploration_times.csv", index=False)